In [ ]:
import pandas as pd
import numpy as np
import scipy as sc
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import FeatureAgglomeration

In [ ]:
df = pd.read_csv('CSV_file')
df_array = df.to_numpy()
# Standardization
scaler_st = StandardScaler()
array_st = scaler_st.fit_transform(df_array)
# make array back to dataframe
df_st = pd.DataFrame(array_st)
df_st.columns = df.columns

In [ ]:
df_feature = df_st.iloc[:,2:30]
df_feature_drop = df_feature.drop(['m_Tmax','w_Tmax','d_Tmax','m_IRmean',
                                   'w_IRmean','d_IRmean','Vineyard','m_Tmean',
                                  'm_RHmean','m_Rtotal','m_WSmean',
                                   'm_IRtotal','Canopy volume'], axis=1)
df_feature_array = df_feature_drop.to_numpy() 

In [ ]:
# Agglomerative clustering features (complete linkage)
def corr_affinity(X): # Method of calculating correlation distance between all sample pairs
    return pairwise_distances(X, metric='correlation') 
clustering = FeatureAgglomeration(distance_threshold=0, n_clusters=None, 
                                  affinity=corr_affinity, linkage='complete') # setting distance_threshold=0 ensures we compute the full tree
clustering_fit = clustering.fit(df_feature_array) # pass the pre-calculated distance matrix for your observations

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram
    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    
# plot dendrogram
pyplot.figure(dpi=600)    
plot_dendrogram(clustering_fit, labels=df_feature_drop.columns, 
                leaf_font_size=8, color_threshold=0.75, 
                distance_sort='ascending', leaf_rotation=45)
pyplot.savefig('plot_file',dpi=600,bbox_inches='tight')
pyplot.show()